In [7]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)

# --- 1. Data Loading and Initial Preparation ---
print("Step 1: Loading and preparing data...")
try:
    # Load the datasets
    df_baskets = pd.read_csv(r"C:\Users\kthac\Desktop\Group_4419\item item similarity\item_item_basket.csv")
    df_counts = pd.read_csv(r"C:\Users\kthac\Desktop\Group_4419\item item similarity\item_item_product_counts.csv")

    print("Data loaded successfully.")
    
    # Ensure data types are correct for memory efficiency
    df_baskets['order_id'] = df_baskets['order_id'].astype('category')
    df_baskets['product_id'] = df_baskets['product_id'].astype('category')

    # Get all unique products and map them to a contiguous index
    product_ids = df_baskets['product_id'].cat.categories
    product_to_index = {product: i for i, product in enumerate(product_ids)}
    index_to_product = {i: product for product, i in product_to_index.items()}
    
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure the CSV files are in the same directory as the script.")
    exit()

# --- 2. Create the Item-Order Sparse Matrix ---
print("\nStep 2: Creating the item-order sparse matrix...")
rows = df_baskets['product_id'].cat.codes
cols = df_baskets['order_id'].cat.codes
data = [1] * len(df_baskets)
item_order_matrix = csr_matrix((data, (rows, cols)),
                               shape=(len(product_ids), len(df_baskets['order_id'].cat.categories)))

print("Shape of the sparse item-order matrix:", item_order_matrix.shape)


# --- 3. Calculate Item-Item Similarity (in Chunks) ---
print("\nStep 3: Calculating item-item similarity matrix...")
co_occurrence_matrix = item_order_matrix.dot(item_order_matrix.T)

actual_count_column = 'count'
# Filter df_counts to only include products present in our sample basket
df_counts_filtered = df_counts[df_counts['product_id'].isin(product_ids)]
product_frequencies = df_counts_filtered.set_index('product_id')[actual_count_column].loc[product_ids].values

sqrt_frequencies = np.sqrt(product_frequencies)

chunk_size = 100
n_products = len(product_ids)
n_chunks = (n_products + chunk_size - 1) // chunk_size

similarity_matrix = lil_matrix((n_products, n_products))

for i in range(n_chunks):
    start_i = i * chunk_size
    end_i = min((i + 1) * chunk_size, n_products)
    
    chunk_co_occurrence = co_occurrence_matrix[start_i:end_i, :].toarray()
    
    chunk_sqrt_freq = sqrt_frequencies[start_i:end_i]
    chunk_denominator = np.outer(chunk_sqrt_freq, sqrt_frequencies)
    chunk_denominator[chunk_denominator == 0] = 1e-9
    
    chunk_similarity = chunk_co_occurrence / chunk_denominator
    similarity_matrix[start_i:end_i, :] = chunk_similarity

print("Shape of the similarity matrix:", similarity_matrix.shape)


# --- 4. Find Top-K Similar Items ---
print("\nStep 4: Finding Top-K similar items for each product...")
K = 10
top_k_similar_items = {}
similarity_matrix_csr = similarity_matrix.tocsr()

for i, product_id in enumerate(product_ids):
    row = similarity_matrix_csr[i].toarray().flatten()
    top_indices = np.argsort(row)[::-1][:K + 1]
    
    similar_products_list = []
    for index in top_indices:
        if index != i:
            similar_product_id = index_to_product[index]
            score = row[index]
            similar_products_list.append({'product_id': similar_product_id, 'score': score})
    
    top_k_similar_items[product_id] = similar_products_list

print("\nTop 3 Similar Items for a sample product (ID: 24852):")
if 24852 in top_k_similar_items:
    top_3 = top_k_similar_items[24852][:3]
    for item in top_3:
        item['score'] = round(item['score'], 5)
    print(top_3)
else:
    print("Sample product ID 24852 not found in the sample data.")


# --- 5. Generate Outputs (Example Usage) ---
print("\nStep 5: Generating candidate items for a user...")

def get_candidate_items(user_recent_purchases, num_recommendations=5):
    candidate_items = {}
    for product_id in user_recent_purchases:
        if product_id in top_k_similar_items:
            for item in top_k_similar_items[product_id]:
                candidate_product = item['product_id']
                score = item['score']
                if candidate_product not in candidate_items or score > candidate_items[candidate_product]:
                    candidate_items[candidate_product] = score
    
    sorted_candidates = sorted(candidate_items.items(), key=lambda x: x[1], reverse=True)
    return [item[0] for item in sorted_candidates[:num_recommendations]]

# Using product IDs known to be popular from your counts file
user_purchases = [13176, 21137] 
recommended_candidates = get_candidate_items(user_purchases, num_recommendations=5)

print(f"\nCandidate items for a user who recently bought products {user_purchases}:")
print(recommended_candidates)
print("\nProcess finished successfully.")

Step 1: Loading and preparing data...
Data loaded successfully.

Step 2: Creating the item-order sparse matrix...
Shape of the sparse item-order matrix: (49677, 3214874)

Step 3: Calculating item-item similarity matrix...
Shape of the similarity matrix: (49677, 49677)

Step 4: Finding Top-K similar items for each product...

Top 3 Similar Items for a sample product (ID: 24852):
[{'product_id': 47766, 'score': 0.18472}, {'product_id': 28204, 'score': 0.16493}, {'product_id': 21137, 'score': 0.15878}]

Step 5: Generating candidate items for a user...

Candidate items for a user who recently bought products [13176, 21137]:
[47209, 21137, 13176, 27966, 21903]

Process finished successfully.


# Step 1: Data Loading and Initial Preparation
This section handles data and initial memory optimization using pandas.
pd.read_csv: Loads the basket and product count data into pandas DataFrames.
.astype('category'): This is a key memory-saving step. Instead of storing product and order IDs as int64 or object strings, they are converted to a category dtype. Pandas internally maps each unique ID to a contiguous integer (accessible via .cat.codes), which is much more memory-efficient and provides a direct lookup for creating our sparse matrix.
ID to Index Mapping: Dictionaries (product_to_index, index_to_product) are created to map the original product_id to its new integer index (0 to n_products-1) and back. This is necessary because the matrix will operate on these zero-based indices.

# Step 2: Creating the Item-Order Sparse Matrix
The goal here is to create a binary matrix representing which items appeared in which orders. A (num_products x num_orders) NumPy array was too large.
scipy.sparse.csr_matrix: We use a Compressed Sparse Row (CSR) matrix because it's highly efficient for both storage and, more importantly, for the fast matrix-vector and matrix-matrix multiplications (.dot() product) required in the next step.
csr_matrix((data, (rows, cols)), shape=...): This is the standard constructor for creating a sparse matrix from coordinate-style data.
rows: The product indices (df_baskets['product_id'].cat.codes).
cols: The order indices (df_baskets['order_id'].cat.codes).
data: A list of ones, indicating the presence of an item in an order.

# Step 3: Calculate Item-Item Similarity 
This is the core of the algorithm. It computes a similarity score between every pair of products.
Co-occurrence Matrix: The line co_occurrence_matrix = item_order_matrix.dot(item_order_matrix.T) is the most critical calculation. By taking the dot product of the matrix with its transpose, we efficiently compute the co-occurrence matrix. The resulting (num_products x num_products) sparse matrix contains the raw count of how many times any two items appeared together in the same basket.
Memory Management: A dense float matrix of the full similarity scores would be too large to fit in memory (~18 GB for 49k products). The code therefore processes the similarity calculation in chunks.
Similarity Metric: The metric used is a normalized co-occurrence. The formula is essentially Co-occurrence(A, B) / (sqrt(Count(A)) * sqrt(Count(B))).
np.outer: The denominator is calculated efficiently for each chunk using NumPy's outer product. This broadcasts the vectors of square-rooted product frequencies, avoiding a slow, explicit Python loop.
lil_matrix: The results are stored in a List of Lists (LIL) matrix. lil_matrix is chosen here because it's efficient for constructing sparse matrices incrementally by changing sparsity structure (i.e., filling in the chunks row by row).

# Step 4: Find Top-K Similar Items
This section extracts the top K most similar items for each product from the final similarity matrix.
.tocsr(): The similarity matrix is converted from LIL back to CSR format. CSR is much more efficient for row slicing, which is exactly what we're about to do in the loop.
np.argsort(): Instead of using a computationally expensive method like sklearn.NearestNeighbors (which would require a dense distance matrix), the code uses a more direct and faster NumPy approach.
np.argsort(row) returns the indices that would sort the array in ascending order.
By reversing this with [::-1], we get the indices of the similarity scores from largest to smallest.
Slicing with [:K+1] gives the indices of the top K+1 items (the item itself plus its K nearest neighbors). This is an extremely fast and memory-efficient way to perform a top-k search on each row.

# Step 5: Generate Outputs (Example Usage)
This final part is a practical demonstration of how to use the pre-computed similarities to generate recommendations.
get_candidate_items function: This function takes a list of a user's recent purchases and returns a ranked list of recommended products.
Candidate Aggregation: It iterates through the user's items and their corresponding similarity lists. A dictionary (candidate_items) is used to aggregate all potential recommendations.
Score Deduplication: Using a dictionary is a clever way to handle duplicates. The line if candidate_product not in candidate_items or score > candidate_items[candidate_product] ensures that if a candidate is suggested by multiple user items, only the highest similarity score is retained.
Final Ranking: Finally, sorted(candidate_items.items(), ...) is used with a lambda function to sort the candidates by their similarity scores in descending order before returning the top N product IDs.